In [98]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import squarify
plt.style.use('seaborn-white')

# Data Preparation

In [99]:
# Creo il dataframe leggendo il file csv
df = pd.read_csv('customer_supermarket.csv', sep='\t')
df.head(3)

,Unnamed: 0,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
0,0,536365,01/12/10 08:26,"2,55",17850.0,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,1,536365,01/12/10 08:26,"3,39",17850.0,United Kingdom,71053,WHITE METAL LANTERN,6
2,2,536365,01/12/10 08:26,"2,75",17850.0,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8


In [100]:
# Elimino la colonna chiamata'Unnamed: 0' perché è superflua (alla lettura del dataset, pandas ha già assegnato un indice
# ad ogni riga)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
0,536365,01/12/10 08:26,"2,55",17850.0,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,01/12/10 08:26,"3,39",17850.0,United Kingdom,71053,WHITE METAL LANTERN,6
2,536365,01/12/10 08:26,"2,75",17850.0,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471910 entries, 0 to 471909
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BasketID         471910 non-null  object 
 1   BasketDate       471910 non-null  object 
 2   Sale             471910 non-null  object 
 3   CustomerID       406830 non-null  float64
 4   CustomerCountry  471910 non-null  object 
 5   ProdID           471910 non-null  object 
 6   ProdDescr        471157 non-null  object 
 7   Qta              471910 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 28.8+ MB


Da df.info() si nota che le colonne CustomerID e ProdDescr hanno alcuni valori NaN.

### Gestione delle righe duplicate

Nel dataframe df sono presenti delle righe duplicate. Dato che il dataset sembra essere un storico degli acquisti fatti su un sito web, posso supporre che i duplicati siano dovuti ad degli errori e quindi possano essere eliminati. Escludo la possibilità che l'utente possa aggiungere al carrello lo stesso prodotto più volte (come acquisti separati). Ad esempio, su Amazon se si prova ad aggiungere al carrello lo stesso oggetto più volte, viene incrementata la quantità dell'oggetto senza aggiungere un nuovo acquisto.

In [102]:
print("Righe duplicate (esclusa la prima occorrenza) all'interno di df: ", df.duplicated().value_counts()[1])

Righe duplicate (esclusa la prima occorrenza) all'interno di df:  5232


In [103]:
df[df.duplicated()].head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
517,536409,01/12/10 11:45,"1,25",17908.0,United Kingdom,21866,UNION JACK FLAG LUGGAGE TAG,1
527,536409,01/12/10 11:45,"2,1",17908.0,United Kingdom,22866,HAND WARMER SCOTTY DOG DESIGN,1
537,536409,01/12/10 11:45,"2,95",17908.0,United Kingdom,22900,SET 2 TEA TOWELS I LOVE LONDON,1


In [104]:
df.drop_duplicates(inplace=True)

Per ogni colonna controllo che i valori siano in formato corretto, diversi da NaN, sensati, ecc.

### CustomerID

Da df.info() si nota che ci sono delle righe con CustomerID = NaN. Dato che l'analisi successiva si concentrerà sull'analisi del comportamento degli utenti, se non è possibile recuperare il CustomerID di queste righe esse andranno eliminate.

Per provare a recuperare questi CustomerID faccio così: per ogni riga $r_1$ con CustomerID = NaN, controllo nel dataframe se esiste una riga $r_2$ con $r_1$.BasketID = $r_2$.BasketID e con $r_2$.CustomerID $\neq$ NaN. Allora, posso assegnare ad $r_1$ il CustomerID di $r_2$.

In [105]:
# null contiene i BasketID delle righe che hanno CustomerID = NaN
null = df[df['CustomerID'].isnull()]['BasketID'].value_counts().index.values

# not_null contiene i BasketID delle righe che hanno CustomerID != NaN
not_null = df[pd.notnull(df['CustomerID'])]['BasketID'].value_counts().index.values

# Questa funzione mi restituisce l'intersezione di due insiemi
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

intersection(list(null), list(not_null))

[]

$null \cap not\_null = \emptyset$. Non è possibile recuperare i valori di CustomerID, quindi elimino le righe che hanno CustomerID = NaN.

In [106]:
df.drop(df[df['CustomerID'].isnull()].index, inplace=True)

### ProdID

Controllando il dataframe, ho notato che alcune righe non rappresentano degli acquisti.

In [107]:
# isalpha restituisce True se la stringa contiene solo caratteri dell'alfabeto
df[df['ProdID'].apply(str.isalpha)]['ProdID'].value_counts()

POST    1197
M        460
D         77
DOT       16
CRUK      16
PADS       4
Name: ProdID, dtype: int64

Le righe che hanno come ProdID uno tra quelli elencati sopra non rappresentano degli acquisti, quindi le elimino.

In [108]:
# Elimino le righe che hanno come ProdID una stringa (quindi solo quelle elencate sopra)
df = df[df['ProdID'].apply(lambda s: not s.isalpha())]

Controllo se ci sono delle righe che non rappresentano degli acquisti ma hanno ProdID alfanumerici (lo faccio perchè con la funzione isalpha un ProdID = '123AAA' non sarebbe stato trovato).

In [109]:
import re

# Questa funzione mi restituisce True se la stringa contiene almeno una lettera dell'alfabeto, False altrimenti
def search_letters(the_string):
    if re.search('[a-zA-Z]', the_string) is None:
        return False
    return True

df[df['ProdID'].apply(search_letters)]['ProdID'].value_counts().index.values

array(['85123A', '85099B', '82494L', '85099C', '85099F', '84997D',
       '84970S', '47591D', '15056N', '84596B', '47590B', '47590A',
       '85049E', '84970L', '84997B', '84029E', '84029G', '47566B',
       '84997C', '85014B', '84596F', '15056BL', '84030E', '85049A',
       '85014A', '16161P', '84406B', '47559B', '85049G', '84997A',
       '84536A', '85049C', '46000S', '47504K', '48173C', '47503A',
       '16156S', '85199S', '84596G', '72351B', '51014A', '16169E',
       '35471D', '47599A', '84510A', '85034C', '75049L', '84509A',
       '15056P', '85184C', '72760B', '85040A', '84032B', '72351A',
       '46000M', '16161U', 'C2', '85132C', '47567B', '72807C', '85061W',
       '84971S', '51014C', '84078A', '79066K', '85231B', '84535B',
       '85035C', '82001S', '47599B', '84032A', '51014L', '85183B',
       '85071B', '84849D', '85049D', '85206A', '84536B', '84279P',
       '72800E', '85032A', '15060B', '47593B', '72802C', '85071A',
       '47504H', '85032B', '72349B', '85034B', '84519B'

Tra i ProdID c'è 'C2', che non rappresenta un acquisto, ma probabilmente una spesa addizionale di spedizione. Quindi elimino le righe in cui ProdID = 'C2.

In [110]:
df[df['ProdID'] == 'C2'].head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
1422,536540,01/12/10 14:05,50,14911.0,EIRE,C2,CARRIAGE,1
10319,537368,06/12/10 12:40,50,14911.0,EIRE,C2,CARRIAGE,1
10652,537378,06/12/10 13:06,50,14911.0,EIRE,C2,CARRIAGE,1


In [111]:
df.drop(df[df['ProdID'] == 'C2'].index, inplace=True)

Tra i ProdID c'è 'BANK CHARGES' che non rappresenta un acquisto, quindi elimino le righe in cui ProdID = 'BANK CHARGES'

In [112]:
df.drop(df[df['ProdID'] == 'BANK CHARGES'].index, inplace=True)

### ProdDescr

Controllo se ci sono ancora delle descrizioni = NaN.

In [116]:
print("Numero di righe con descrizione NaN: ", df[df['ProdDescr'].isnull()].shape[0])

Numero di righe con descrizione NaN:  0


### BasketDate

Controllo che tutti i valori della colonna BasketDate siano delle stringhe che rappresentano correttamente delle date.

In [117]:
from dateutil.parser import parse

# Questa funzione controlla se la stringa presa in input rappresenta correttamente una data
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

# Controllo se tutte i valori di BasketDate sono delle date valide
print(df['BasketDate'].apply(is_date).value_counts())

True    399689
Name: BasketDate, dtype: int64


Tutti i valori rappresentano delle date. Ora controllo se questi valori sono scritti nello stesso formato.

In [118]:
import datetime

def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%d/%m/%y %H:%M')
        return True
    except ValueError:
        return False
    
print(df['BasketDate'].apply(validate).value_counts())

True    399689
Name: BasketDate, dtype: int64


Tutti i valori sono scritti seguendo lo stesso formato (giorno/mese/anno ora:minuto). Infine, converto tutte queste stringhe in formato datetime.

In [119]:
df['BasketDate'] = pd.to_datetime(df['BasketDate'], format="%d/%m/%y %H:%M")

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399689 entries, 0 to 471908
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   BasketID         399689 non-null  object        
 1   BasketDate       399689 non-null  datetime64[ns]
 2   Sale             399689 non-null  object        
 3   CustomerID       399689 non-null  float64       
 4   CustomerCountry  399689 non-null  object        
 5   ProdID           399689 non-null  object        
 6   ProdDescr        399689 non-null  object        
 7   Qta              399689 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.4+ MB


### Sale

I valori della colonna Sale sono delle stringhe che rappresentano dei float, dove al posto del punto è stata usata la virgola.

In [121]:
# Questa funzione converte una stringa nel rispettivo float (3,5 --> 3.5)
def convert_to_float(s):
    comma_pos = s.find(',')
    if comma_pos >= 0:
        s = s[: comma_pos] + '.' + s[comma_pos+1 :]
    return float(s)

df['Sale'] = df['Sale'].apply(convert_to_float)

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399689 entries, 0 to 471908
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   BasketID         399689 non-null  object        
 1   BasketDate       399689 non-null  datetime64[ns]
 2   Sale             399689 non-null  float64       
 3   CustomerID       399689 non-null  float64       
 4   CustomerCountry  399689 non-null  object        
 5   ProdID           399689 non-null  object        
 6   ProdDescr        399689 non-null  object        
 7   Qta              399689 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.4+ MB


Tutte i valori della colonna Sale sono stati convertiti con successo a float64.

Controllo se ci sono delle righe con Sale $\leq$ 0.

In [135]:
df[df['Sale'] <= 0].head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta


In [129]:
print("Numero di righe con Sale <= 0: ", df[df['Sale'] <= 0].shape[0])

Numero di righe con Sale <= 0:  33


In [131]:
print("Numero di righe con Sale = 0: ", df[df['Sale'] == 0].shape[0])

Numero di righe con Sale = 0:  33


Non ci sono righe con Sale < 0. Mentre le righe con Sale = 0 potrebbero riferirsi a dei prodotti omaggio. Io sono interessato agli acquisti, quindi posso eliminare queste righe.

In [133]:
df.drop(df[df['Sale'] == 0].index, inplace=True)

### CustomerCountry

Dal df.info() iniziale si vede che non ci sono valori di CustomerCountry = NaN. Tuttavia, ci sono delle righe con CustomerCountry = 'Unspecified'.

In [66]:
print("Righe con CustomerCountry = 'Unspecified': ", df[df['CustomerCountry'] == 'Unspecified'].shape[0])

Righe con CustomerCountry = 'Unspecified':  244


In [137]:
df[df['CustomerCountry'] == 'Unspecified'].head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
128523,549687,2011-04-11 13:29:00,7.95,12363.0,Unspecified,20685,DOORMAT RED RETROSPOT,2
128524,549687,2011-04-11 13:29:00,7.95,12363.0,Unspecified,22691,DOORMAT WELCOME SUNRISE,2
128525,549687,2011-04-11 13:29:00,7.95,12363.0,Unspecified,48116,DOORMAT MULTICOLOUR STRIPE,2


CustomerCountry = 'Unspecified' potrebbe essere un valore corretto, in quanto l'utente in fase di acquisto non ha voluto specificare lo stato in cui abita (ad esempio per motivi di privacy) e quindi lascio questi valori nel dataframe.

### Qta e BasketID

Controllo se ci sono delle righe che hanno Qta $\leq$ 0.

In [146]:
print("Numero di righe con Qta <= 0: ", df[df['Qta'] <= 0].shape[0])

Numero di righe con Qta <= 0:  8506


In [147]:
print("Numero di righe con Qta <= 0: ", df[df['Qta'] < 0].shape[0])

Numero di righe con Qta <= 0:  8506


Non ci sono righe con Qta = 0.

In [148]:
df[df['Qta'] < 0].head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
154,C536383,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,35004C,SET OF 3 COLOURED FLYING DUCKS,-1
235,C536391,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,22556,PLASTERS IN TIN CIRCUS PARADE,-12
236,C536391,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,21984,PACK OF 12 PINK PAISLEY TISSUES,-24


Controllo se tutte le righe con Qta negativa hanno il BaskeID che inizia con la lettera C.

In [141]:
import re

# Questa funziona mi restituisce True se la stringa presa in input contiene un carattere dell'alfabeto, False altrimenti.
def search_letters(the_string):
    if re.search('[a-zA-Z]', the_string) is None:
        return False
    return True

df[df['Qta'] <= 0]['BasketID'].apply(search_letters).value_counts()


True    8506
Name: BasketID, dtype: int64

Tutte le righe con Qta negativa hanno BasketID che inizia con la lettera C. Queste righe sembrano rappresentare degli ordini annullati o dei rimborsi.

In [ ]:
# GESTIRE I RIMBORSI!!!!!!

### BasketID

BasketID sono di tipo object, quindi controllo se ci sono BasketID alfanumerici. Inoltre da df.info() si nota che non ci sono BasketID = NaN.

In [68]:
import re
# Questa funziona mi restituisce True se la stringa presa in input contiene un carattere dell'alfabeto, False altrimenti.
def search_letters(the_string):
    if re.search('[a-zA-Z]', the_string) is None:
        return False
    return True

# Salvo sul dataframe tmp le righe (del dataframe df) che hanno BasketID alfanumerico
tmp = df[df['BasketID'].apply(search_letters)]
print("Numero di righe che hanno BaskedID alfanumerico:", tmp.shape[0])

Numero di righe che hanno BaskedID alfanumerico: 8539


In [69]:
tmp.head(3)

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
154,C536383,2010-12-01 09:49:00,"4,65",15311.0,United Kingdom,35004C,SET OF 3 COLOURED FLYING DUCKS,-1
235,C536391,2010-12-01 10:24:00,"1,65",17548.0,United Kingdom,22556,PLASTERS IN TIN CIRCUS PARADE,-12
236,C536391,2010-12-01 10:24:00,"0,29",17548.0,United Kingdom,21984,PACK OF 12 PINK PAISLEY TISSUES,-24


Le righe in cui BasketID inizia con la lettera C sembrano rappresentare una cancellazione di un ordine, ossia una rimborso.
Prima di gestire tali rimborsi controllo se ci sono altre righe che hanno BasketID alfanumerico, ma senza la lettera 'C'.

In [70]:
def search_letters_not_C(the_string):
    if re.search('[a-zA-Z]', the_string) is None:
        return False
    elif re.search('[C]', the_string) is None:
        return True
    return False

tmp[tmp['BasketID'].apply(search_letters_not_C)]

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta


Queste due righe hanno CustomerID = NaN. Provo a cercare se all'interno del dataset ci sono altre righe aventi lo stesso BasketID per provare recuperare il valore di CustomerID (se due righe hanno lo stesso BasketID allora hanno anche lo stesso CustomerID).

In [71]:
df[(df['BasketID'] == 'A563186') | (df['BasketID'] == 'A563187')]

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta


Ritrovo solo le due righe con BasketID = NaN, quindi non posso recuperare il CustomerID e le elimino.

In [72]:
# Elimino le due righe dal dataframe df
df = df[(df['BasketID'] != 'A563186') & (df['BasketID'] != 'A563187')]
# Elimino le due righe dal dataframe tmp, lo faccio perché uso tmp anche dopo
tmp = tmp[(tmp['BasketID'] != 'A563186') & (tmp['BasketID'] != 'A563187')]

Per gestire i rimborsi, devo prima toglierli dal dataframe df.

In [73]:
# Elimino le righe che contengono delle lettere nel BasketID, osssia tutti i rimborsi (tanto sono salvati sul dataframe tmp)
df = df[df['BasketID'].apply(lambda x: not search_letters(x))]

In [74]:
print("Numero di rimborsi: ", tmp.shape[0])

Numero di rimborsi:  8539


Occorre ora gestire i rimborsi, ossia le righe del dataframe tmp. I rimborsi vengono gestiti facendo le seguenti assunzioni:
* Per ogni ordine è possibile richiedere al massimo un rimborso.
* Un rimborso può essere applicato ad un ordine solo se la data dell'ordine è antecedente a quella del rimborso, il codice prodotto è lo stesso, il codice cliente è lo stesso e la quantità di oggetti presente nell'ordine è maggiore o uguale alla quantità in valore assoluto degli oggetti indicata nel rimborso.
* Dato un rimborso, se all'interno del dataset non è presente un ordine antecedente a quel rimborso, allora il rimborso non viene considerato.
* Dato un ordine, se ci sono più rimborsi applicabili a questo ordine, si applica quello con la quantità di oggetti in valore assoluto maggiore. Se ci sono due o più rimborsi con la stessa quantità ne uso uno tra questi.

In [75]:
"""
def update(row, tmp):
    basket_date = pd.to_datetime(row['BasketDate'])
    customer_id = row['CustomerID']
    prod_id = row['ProdID']
    qta = row['Qta']
    r = tmp[(tmp['ProdID'] == prod_id) & (tmp['CustomerID'] == customer_id) & (pd.to_datetime(tmp['BasketDate']) >= basket_date) & (abs(tmp['Qta']) <= qta )]
    if r.shape[0] <= 0:
        return row
    qta_min = r['Qta'].min()
    # Mi interessano solo i rimborsi che hanno Qta = qta_min e tra questi prendo il primo che trovo
    rimborso = r[r['Qta'] == qta_min].iloc[0]
    basket_id_rimborso = rimborso['BasketID']
    prod_id_rimborso = rimborso['ProdID']
    qta_rimborso = rimborso['Qta']
    # Elimino da tmp la riga che corrisponde a rimborso
    tmp.drop(tmp[(tmp['BasketID'] == basket_id_rimborso) & (tmp['ProdID'] == prod_id_rimborso)].index[0], inplace=True)
    # Aggiorno il valore di Qta
    row['Qta'] = qta + rimborso['Qta']
    print(tmp.shape[0])
    return row
    
prova = df.apply(lambda row: update(row, tmp), axis=1)
"""

"\ndef update(row, tmp):\n    basket_date = pd.to_datetime(row['BasketDate'])\n    customer_id = row['CustomerID']\n    prod_id = row['ProdID']\n    qta = row['Qta']\n    r = tmp[(tmp['ProdID'] == prod_id) & (tmp['CustomerID'] == customer_id) & (pd.to_datetime(tmp['BasketDate']) >= basket_date) & (abs(tmp['Qta']) <= qta )]\n    if r.shape[0] <= 0:\n        return row\n    qta_min = r['Qta'].min()\n    # Mi interessano solo i rimborsi che hanno Qta = qta_min e tra questi prendo il primo che trovo\n    rimborso = r[r['Qta'] == qta_min].iloc[0]\n    basket_id_rimborso = rimborso['BasketID']\n    prod_id_rimborso = rimborso['ProdID']\n    qta_rimborso = rimborso['Qta']\n    # Elimino da tmp la riga che corrisponde a rimborso\n    tmp.drop(tmp[(tmp['BasketID'] == basket_id_rimborso) & (tmp['ProdID'] == prod_id_rimborso)].index[0], inplace=True)\n    # Aggiorno il valore di Qta\n    row['Qta'] = qta + rimborso['Qta']\n    print(tmp.shape[0])\n    return row\n    \nprova = df.apply(lambda row

Dopo aver applicato i rimborsi, bisogna togliere dal dataframe df tutti gli ordini che hanno Qta = 0, ossia il cliente ha mandato indietro tutti i prodotti che aveva acquistato, annullando quindi l'ordine.

In [76]:
df.drop(df[df['Qta'] == 0].index, inplace=True)

Nel dataframe df tutti i valori di BasketID sono delle stringhe che contengono solo numeri, quindi posso convertirle in interi.

In [77]:
df['BasketID'] = df['BasketID'].astype("int64")